In [80]:
import requests, re
from bs4 import BeautifulSoup

In [123]:
def url_gen(ref,servings=1, metric = 'true', typeSearch = 'ingredients'):
    if typeSearch == 'none':
        tail = ''
    else :
        if typeSearch == 'nutrition':
            tail = 'fullrecipenutrition/'
        else:  
            tail = 'print/?recipeType=Recipe&servings={}&isMetric={}'.format(servings,metric)

    return ('https://www.allrecipes.com/recipe/' + ref+tail)

In [124]:
def get_ingredients(ref): 
    
    url = url_gen(ref)
    r=requests.get(url)
    
    ingredients = []
    soup = BeautifulSoup(r.text.encode('utf-8'),'lxml')    
    col1 = soup.find('h2').next_sibling.next_sibling
    col2 = col1.next_sibling.next_sibling

    for i in col1.findAll('li'):
        last=str(i.contents).rfind('r')
        first=str(i.contents).find('n')
        ingredients.append((str(i.contents)[(first+1):(last-1)].lstrip()))
    for i in col2.findAll('li'):
        last=str(i.contents).rfind('r')
        first=str(i.contents).find('n')
        ingredients.append((str(i.contents)[(first+1):(last-1)].lstrip()))
    return ingredients

In [125]:
def get_popularity(ref):    
    url = url_gen(ref,typeSearch = 'none')
    r=requests.get(url)
    soup = BeautifulSoup(r.text.encode('utf-8'),'lxml')    
    step1 =str(soup.findAll('div', class_="total-made-it"))

    begin = '"made-it-count"></span><span>'
    end = '\xa0made it'
    return step1[step1.find(begin)+len(begin):step1.find(end)]     #3k --> 3000

In [126]:
def get_nutrition(ref):
#Returns a dictionnary with nutritional values

    url = url_gen(ref, typeSearch = 'nutrition')
    r=requests.get(url)
    soup = BeautifulSoup(r.text.encode('utf-8'),'lxml') 
    info = soup.findAll('div', class_='nutrition-row')
    general = {}
    general['Title']  = str(soup.find('h2').contents[0])
    general['Reference'] = ref
    general['Region'] = 'Unknown'
    nutrition = {}
    getCal = str(soup.find('div', class_="nutrition-top light-underline"))
    key = 'Calories:</span> '
    nutrition['Calories'] = int(getCal[getCal.find(key) + len(key):getCal.rfind('<br/>')])
    
    for i in info:
        a=i.find(class_='nutrient-name')
        b = re.findall('>(.*?):', str(a))[0]
        c = re.findall('value">(.*?)<', str(a))[0]
       # print(str(b) + ' : ' + str(c))
        nutrition[b] = c
    return general, nutrition

In [127]:
class Recipe:
    def __init__(self, name):
        self.name = name
        self.ingredients = get_ingredients(name)
        self.general, self.nutrition = get_nutrition(name)
        self.general['Popularity'] = get_popularity(name)
    def display(self):
        print('\n ----------------- \n')
        print(self.general)
        print('\n')
        print(self.ingredients)
        print('\n')
        print(self.nutrition)
    def toDict(self):
        dictio = {}
        dictio['General'] = self.general
        dictio['Nutrition'] = self.nutrition
        dictio['Ingredients'] = self.ingredients
        return dictio

In [128]:
ref = '246179/black-chana-with-potato/'
ref = '45736/chicken-tikka-masala/'
#get_ingredients(ref)
a = Recipe(ref)
a.toDict()

{'General': {'Title': 'Chicken Tikka Masala',
  'Reference': '45736/chicken-tikka-masala/',
  'Region': 'Unknown',
  'Popularity': '3k'},
 'Nutrition': {'Calories': 404,
  'Total Fat': '28.9g',
  'Saturated Fat': '17.0g',
  'Cholesterol': '143mg',
  'Sodium': '1592mg',
  'Potassium': '660mg',
  'Total Carbohydrates': '13.3g',
  'Dietary Fiber': '2.5g',
  'Protein': '24.6g',
  'Sugars': '7g',
  'Vitamin A': '2432IU',
  'Vitamin C': '11mg',
  'Calcium': '206mg',
  'Iron': '3mg',
  'Thiamin': '0mg',
  'Niacin': '13mg',
  'Vitamin B6': '1mg',
  'Magnesium': '57mg',
  'Folate': '24mcg'},
 'Ingredients': ['60 ml yogurt',
  '4 ml lemon juice',
  '1 g fresh ground cumin',
  '0.6 g ground cinnamon',
  '0.9 g cayenne pepper',
  '1 g freshly ground black pepper',
  '1 g minced fresh ginger',
  '2 g salt, or to taste',
  '3/4 boneless skinless chicken breasts, cut into bite-size pieces',
  '1 long skewers',
  '4 g butter',
  '1/4 clove garlic, minced',
  '1/4 jalapeno pepper, finely chopped',
  '1

In [129]:
def get_recipes(themeUrl, number = 20):
    
    r=requests.get(themeUrl) #add page 1,2,etc
    
    baseUrl = 'https://www.allrecipes.com/recipe/'
    soup = BeautifulSoup(r.text.encode('utf-8'),'lxml') 
    results = soup.findAll('h3', class_='fixed-recipe-card__h3')
    
    theme = []
    iteration = 0
    for i in results :
        
        if iteration  == number:
            break
            
        iteration = iteration + 1
        
        link = str(i.find('a').get('href'))
        first=link.find(baseUrl)
        theme.append(link[(first+len(baseUrl)):])
        
    return theme

In [130]:
def getThemeRecipes(themeName, number = 20):
    tail = ''
    for i in range(1,len(themeName)-1):
        tail = tail + themeName[i]+'/'
    themeUrl = 'https://www.allrecipes.com/recipes/'+themeName[0]+'/world-cuisine/'+tail
    themeLinks = get_recipes(themeUrl, number = number)
    themeRecipes = []
    themeDico = {}

    from tqdm import tqdm_notebook as tqdm

    for i in tqdm(range(len(themeLinks))):
        recipe = themeLinks[i]
    #for recipe in themeLinks:
        try:
            current = Recipe(recipe)
            themeRecipes.append(current)
            themeDico[current.general['Reference']] = current.toDict()
            themeDico[current.general['Region']]    = themeName[len(themeName)-1]
        except AttributeError:
            print('Error')
            continue
    return themeDico

In [131]:
themeIndian = ['233','asian','indian']
themeMexican = ['728','latin-american','mexican']

themeDico = getThemeRecipes(themeMexican)

In [132]:
import json

with open(themeName[len(themeName)-1]+'.json', 'w') as f:
    json.dump(themeDico, f)